# Machine Learning

Machine learning is the science (and art) of programming computers so they can learn from data. 

For example:
- Ranking your web search result (Google)
- Recommendation system: Amazon
- AlphaGo
- Chatgpt or Large Language Model (Large Language Model)
- Spam filter
- Image classification
- Detecting credit card fraud
- ...

In general, we hope to use previous data to design(train) some algorithms(model), then use trained models to predict when we receive new data.

- Example 1: We can use emails (data) to construct a spam filter (algorithm, model), then we can use spam filter to predict that new emails (new data) is a spam or not.

- Example 2: We can use your current friends list (data) to construct a friend recommendation system (algorithm, model), then we can use this model to find people (new data) who you may know.

- ...

So, our goal is to finding a model that can be used on **unseen data**.


## Machine Learning pipeline:

1. Look at the Big Picture:
    
    - You should understand your task, e.g regression (predict housing price) or classification (handwritten numbers).
    - Select a performance measure
    - ...

2. Get the Data

    - Download the data from internet
    - ...

3. Discover your data and visualize your data

    - Pandas and Matplotlib
    - ...

4. Prepare your data for Machine Learning Algorithm

    - Algorithm only accepts numeric values
    - Data cleaning (fillna, dropna)
    - Text and categorical attributes to numerics
    - ...

5. Select and train your model


6. Fine-tune your model

    - Cross-validation
    - try different models and ensemble them
    - evaluate your model on test set

7. Launch, monitor, and maintain your model



# Supervised learning

Supervised learning means that you can use both input $x$ and output $y$ to train a model. The output $y$ is also called *labels*.

If the labels are "continuous" real numbers (e.g. stock price, house price, insurance fee, height, weights, and etc ), then it is **regression problem**.

If the labels are discrete categories (e.g. dog vs cat, True vs False, handwritten digits numbers(0-9), 0 vs 1, or -1 vs 1, and etc), then it is **classification problem**.


We will also discuss unsupervised learning and semisupervised learning problem.


## Why do we do train/test split?

- Remember that our goal is to create a model such that it can be generalized to unseen data set.

- It is easy to design some trivial models that performs well on training data set but fail to work on new data points. This phenomenon is called **overfitting**.


## No Free Luch (NFL) theorem:

A model is a simplified version of the observations. The simplifications are meants to discard the superfluous details that are unlikely to generalize to new instances. To decide what data to discard and what data to keep, you must make *assumptions*. For example, a linear model makes the assumption that the data is fundamentally linear and that the distance between instances and the straight line is just noise, which can safely be ignored.

In a famous 1996 paper, see [here](https://direct.mit.edu/neco/article/8/7/1341/6016/The-Lack-of-A-Priori-Distinctions-Between-Learning), David Wolpert demonstrated that if you make no assumption about the data, then there is no reason to prefer one model over any other. This is called the No Free Lunch (NFL) theorem. For some datasets the best model is linear model, while for other datasets it is a neural network. There is no model that is a priori guaranteed to work better. The only way to know for sure which model is to evaluate them all. 

Since this is not possible, in practice you make some reasonable assumptions (based on visualization or data preprocessing) and evaluate only a few reasonable models. For example, for simple tasks you may evaluate linear models with various levels of regularization, and for a complex problem you may evaluate various neural networks.

In [34]:
import numpy as np
import matplotlib.pyplot as plt

## Regression example

In this part, we make a fake regression dataset and then train a linear regression model. The main goal is to review the basic machine learning process:

Generating fake dataset:

    sklearn.datasets.make_regression(n_samples=100, n_features=100, *, n_informative=10, n_targets=1, bias=0.0, effective_rank=None, tail_strength=0.5, noise=0.0, shuffle=True, coef=False, random_state=None)

In [53]:
from sklearn.datasets import make_regression
X, Y = make_regression(n_samples=20000, n_features = 100, noise=1)
print(X.shape, Y.shape)

(20000, 100) (20000,)


Train test split

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

Usually, we need to normalize the training data, like what we did in PCA. I skip this step here because I am using randomly generated data and each column are from the same distribution or in the same scale. 

After normalization, we can train the model, evaluate the model on the test data.

In [55]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# fit model 
reg = LinearRegression().fit(X_train, y_train)

# evaluate on the test data
y_pred = reg.predict(X_test)

# Since it is a regression problem, mean squared error is a common metric to evaluate the performance
# you can use built-in command to compute the error
mse1 = mean_squared_error(y_test, y_pred)
print(f'MSE is {mse1:.5f}')

# or you can use your own code
mse2 = np.linalg.norm(y_pred-y_test)**2/np.size(y_test)
print(f'MSE is {mse2:.5f}')


MSE is 1.01203
MSE is 1.01203


# Feature selection

Notice that there are ten informative features, let's try to select them.

In [64]:
# count the number of informative features
print( np.sum(reg.coef_>=0.1) )

# select the index
idx = np.argwhere(reg.coef_>=0.1).squeeze()

# define a new training and test dataset
X_train_reduced = X_train[:,idx]
X_test_reduced = X_test[:,idx]

# fit model 
reg1 = LinearRegression().fit(X_train_reduced, y_train)

# evaluate on the test data
y_pred = reg1.predict(X_test_reduced)

# compute error
mse1 = mean_squared_error(y_test, y_pred)
print(f'MSE is {mse1:.5f}')

# or you can use your own code
mse2 = np.linalg.norm(y_pred-y_test)**2/np.size(y_test)
print(f'MSE is {mse2:.5f}')


10
MSE is 1.00215
MSE is 1.00215


# Classification example

We can use the `make_classification` command to generate synthetic data samples.

    sklearn.datasets.make_classification(n_samples=100, n_features=20, *, n_informative=2, n_redundant=2, n_repeated=0, n_classes=2, n_clusters_per_class=2, weights=None, flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0, scale=1.0, shuffle=True, random_state=None)

In [6]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, Y = make_classification(n_samples=10000, n_features = 30, n_informative=6, n_classes=4)
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

## Model 1: K-nearest neighbors

#### Algorithm explanation:

An object is classified by a plurality vote of its neighbors, with the object being assigned to the class most common among its k nearest neighbors (k is a positive integer, typically small). If k = 1, then the object is simply assigned to the class of that single nearest neighbor.

#### Regression:
K-nearest neighbors algorithm can also be used for regression problem. In k-NN regression, the output is the property value for the object. This value is the average of the values of k nearest neighbors. If k = 1, then the output is simply assigned to the value of that single nearest neighbor.

#### Documentation:
Python documentation: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [10]:
from sklearn.neighbors import KNeighborsClassifier

# train the model
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_train, y_train)

# test accuracy
y_pred = neigh.predict(X_test)
acc = neigh.score(X_test, y_test)
print(f'Test accuracy is {acc:.2f}')

# compute test accuracy without using score
acc1 = np.sum(y_pred.reshape(y_test.shape) == y_test) / np.size(y_test)
print(f'Test accuracy is {acc1:.2f}')

# return probability
prob = neigh.predict_proba(X_test)
print(prob)

Test accuracy is 0.74
Test accuracy is 0.74
[[0.625 0.125 0.125 0.125]
 [0.75  0.25  0.    0.   ]
 [0.    0.    1.    0.   ]
 ...
 [0.125 0.5   0.25  0.125]
 [0.625 0.    0.    0.375]
 [0.125 0.625 0.    0.25 ]]


You can use cross-validation to select the best number of neighbors.

## Logistic Regression

See wikipedia for model explanation: https://en.wikipedia.org/wiki/Logistic_regression

**Logistic regression is designed for classification problems only. In other words, you can not use this model for regression problems.**

Python documentation: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [23]:
# train the model
from sklearn.linear_model import LogisticRegression

Log = LogisticRegression().fit(X_train, y_train)

# test accuracy
y_pred = Log.predict(X_test)
acc = Log.score(X_test, y_test)
print(f'Test accuracy is {acc:.2f}')

# compute test accuracy without using score
acc1 = np.sum(y_pred.reshape(y_test.shape) == y_test) / np.size(y_test)
print(f'Test accuracy is {acc1:.2f}')

# return probability
prob = Log.predict_proba(X_test)
print(prob)

Test accuracy is 0.59
Test accuracy is 0.59
[[0.56281303 0.04135957 0.07570028 0.32012711]
 [0.20904043 0.51212522 0.15466231 0.12417203]
 [0.67553186 0.12558172 0.16790161 0.03098482]
 ...
 [0.22122758 0.59547218 0.11085617 0.07244407]
 [0.79096746 0.02564928 0.00510108 0.17828218]
 [0.07873112 0.75625907 0.1178367  0.04717311]]


## Tree based model

A decision tree looks like this: ![](https://scikit-learn.org/stable/_images/sphx_glr_plot_iris_dtc_002.png)


A decision tree is a flowchart-like structure in which each internal node represents a "test" on an attribute (e.g. whether a coin flip comes up heads or tails), each branch represents the outcome of the test, and each leaf node represents a class label (decision taken after computing all attributes). The paths from root to leaf represent classification rules.

Python documentation: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [32]:
from sklearn.tree import DecisionTreeClassifier

Tree = DecisionTreeClassifier(max_depth=10).fit(X_train, y_train)

# test accuracy
y_pred = Tree.predict(X_test)
acc = Tree.score(X_test, y_test)
print(f'Test accuracy is {acc:.2f}')

# compute test accuracy without using score
acc1 = np.sum(y_pred.reshape(y_test.shape) == y_test) / np.size(y_test)
print(f'Test accuracy is {acc1:.2f}')

# return probability
prob = Tree.predict_proba(X_test)
print(prob)

Test accuracy is 0.76
Test accuracy is 0.76
[[0.65454545 0.         0.23636364 0.10909091]
 [0.75       0.         0.         0.25      ]
 [0.         0.         1.         0.        ]
 ...
 [0.03688525 0.91803279 0.03278689 0.01229508]
 [0.06976744 0.06976744 0.         0.86046512]
 [0.03688525 0.91803279 0.03278689 0.01229508]]


## Ensemble Learning

To better understand random forest model, we should know **Ensemble Learning** first, which is an important technique in Machine Learning.

#### Motivation:

Suppose that you have a complex question of thousands of random people, then aggregate their answers. In many cases you will find yjay yhis aggregated answer is better than an expert's answer. This is called the *wisdom of the crowd*. Similarly, if you aggregate the predictions of a groups of predictors (such as classifiers or regressors), you will often get better predictions than with the best individual predictor. A group of predictors is called an ensemble; thus this technique is called Ensemble Learning.

#### When do we use ensemble model?

Usually, ensemble model works better than single model, but there is no guarantee. In my opinion, since we must try different single models due to No Free Luch Theorem, it does not hurt to ensemble all single models you have tried, and look at the performance on test dataset.

#### Example:

Let's ensemble KNNeighbor, Logistic Regression, and Tree model together.

Python documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html

In [60]:
from sklearn.ensemble import VotingClassifier

# define classifiers you want to use
kn_clf = KNeighborsClassifier()
log_clf = LogisticRegression()
tree_clf = DecisionTreeClassifier()

# define max vote classifier
voting_clf = VotingClassifier( estimators=[('kn',kn_clf),('lr',log_clf),('tree',tree_clf)],
                             voting='hard')
# train max vote classifier
voting_clf.fit(X_train, y_train)

# let's look at each classifier's accuracy on the test set:
for clf in (kn_clf, log_clf, tree_clf, voting_clf):
    
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    print(clf.__class__.__name__, acc)

KNeighborsClassifier 0.7235
LogisticRegression 0.588
DecisionTreeClassifier 0.71
VotingClassifier 0.741


## Random Forest

In short, random forest is an ensemble of decision trees. 

A group of Decision Tree classifiers are trained on a different random subset of the training set. Then, you use max vote (ensemble technique) to obtain the prediction. 


Python documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


In [63]:
from sklearn.ensemble import RandomForestClassifier

# train random feature model
rf = RandomForestClassifier().fit(X_train, y_train)

# accuracy
print( f'Test accuracy is {rf.score(X_test, y_test)}' )

# return probability
prob = rf.predict_proba(X_test)
print(prob)

Test accuracy is 0.8245
